## ASSUMPTION
This example is written assuming we have a service up and running for us to utilise

Copyright IBM Corp. 2022, 2023
@author James Strudwick IBM Research

## Dependencies

In [1]:
#For Sterimol
from morfeus import Sterimol, read_xyz

In [2]:
#For descriptors
import pandas as pd
import numpy as np
from rdkit.Chem import AllChem, Descriptors, MolFromSmiles
from mordred import Calculator, descriptors, error

In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem

## Data recording

In [4]:
# SMILES from cheminfo.org and pubchem: propane reactant, pentane product, side product, solvent, HFIP, acid catalyst
MCP_SMILES=[r'C1(/C=C2CC\2)=CC=CC=C1', 
r'BrC(C=C1)=CC=C1/C=C2CC\2', 
r'ClC(C=C1)=CC=C1/C=C2CC\2', 
r'FC(C=C1)=CC=C1/C=C2CC\2', 
r'O=[N+](C(C=C1)=CC=C1/C=C2CC\2)[O-]',
r'COC(C=C1)=CC=C1/C=C2CC\2',
r'CN(C(C=C1)=CC=C1/C=C2CC\2)C',
r'CC(C)C(C=C1)=CC=C1/C=C2CC\2',
r'C1(/C=C2CC\2)=CC=C(C3=CC=CC=C3)C=C1',
r'CC1=CC(/C=C2CC\2)=CC=C1',
r'BrC1=CC(/C=C2CC\2)=CC=C1',
r'CC1=CC=CC=C1/C=C2CC\2',
r'CC(C)C1=CC(/C=C2CC\2)=CC=C1',
r'FC(C(C=C1)=CC=C1/C=C2CC\2)(F)F',
r'CC(C=C1)=CC=C1/C=C2CC\2']
Catalyst_SMILES=['CC1=CC=C(S(O)(=O)=O)C=C1',
'O=S(C1=CC=CC=C1)(O)=O',
'CS(=O)(O)=O',
'[H]Cl',
'F[B-](F)(F)F.[H+]',
'CC1(C)[C@]2([H])CC[C@@]1(CS(O)(=O)=O)C(C2)=O',
'OC1=C(F)C(F)=C(F)C(F)=C1F','O=S(O)(C(F)(F)F)=O',
'[H]Br','F[P-](F)(F)(F)(F)F.[H+]',
'O=C(O)C(F)(F)F',
'O=P(O)(O)O',
'O=Cl(=O)(O)=O',
'[H]I',
'O=P(OC1=CC=CC=C1)(OC2=CC=CC=C2)O',
'O=S(NS(=O)(C(F)(F)F)=O)(C(F)(F)F)=O',
'O=S(C1=CC=CC=C1C2=CN=NC(C3=NC=C(C4=CC=CC=C4S(=O)([O-])=O)C=C3)=N2)([O-])=O.[Na+].[Na+]',
'CCCC[N+](CCCC)(CCCC)CCCC.[O-]S(=O)(O)=O',
'O=P(OC1=CC=C2C(C=CC=C2)=C31)(O)OC4=C3C(C=CC=C5)=C5C=C4']
solvent_SMILES=['ClC(Cl)Cl',
                'ClCCCl',
                'ClCCl',
                'CCC(C)(O)C',
                'CC1=CC=CC=C1',
                'CC1=CC(C)=CC(C)=C1',
                'FC(F)(F)C1=CC=CC=C1',
                'FC1=C(F)C(F)=C(F)C(F)=C1F',
                'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
                'C[N+]([O-])=O',
                '[H]O[H]',
                'C1CCOC1',
                'O1CCOCC1']
cosolvent_SMILES=['FC(F)(F)C(O)C(F)(F)F',
                  'F/C(F)=C(F)\F',
                  'FC(F)(F)C(C(F)(F)F)(O)C(F)(F)F',
                  'CC(C)(O)C',
                  'CC(O)C']
additives_SMILES=['O=S([O-])([O-])=O.[Na+].[Na+]',
                  'CC(C1=CC(C)=CC(C(C)(C)C)=C1O)(C)C',
                  '[H]O[H]','[O-][Si](=O)[O-].[O-][Si](=O)[O-].[Na+].[Al+3]',
                  'O=C1CC[C@@]2([H])[C@]3([H])CC=C4C[C@@H](OC(C)=O)CC[C@]4(C)[C@@]3([H])CC[C@]12C',
                  'CO','O1CCOCC1']
# Initiate list of rdkit molecules
rdkit_mols = [MolFromSmiles(smiles) for smiles in MCP_SMILES]

## Sterimol

In [5]:
list_steri=[]
for i in range(len(MCP_SMILES)):
#     try:
    mol = Chem.MolFromSmiles(MCP_SMILES[i])  
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol)
    Chem.MolToXYZFile(mol,str(i))

    elements, coordinates = read_xyz(str(i))
    sterimol = Sterimol(elements, coordinates, 1, 2)
    B1=sterimol.B_1_value
    B5=sterimol.B_5_value
    bond_length=sterimol.bond_length
    L=sterimol.L_value
    list_steri.append([B1,B5,bond_length,L])
#     except OSError:
#         print('skip',i)
#         continue

In [8]:
MCP_df=pd.DataFrame(list_steri,columns=['B1','B5','bond_length','L']).to_csv('MCP_sterimol.csv')